#### **Rivaldo Lumelino, Alexandr Voronovich**
#### **CSC 36000 – Final Project**

#### **Problem 1**
##### **How can agents communicate and coordinate when network is poor (urban congestion or rural coverage gaps)?**
##### **Without solution → collisions, inefficiency, unsafe behavior**

In [5]:
import random

In [6]:
import math

In [7]:
from collections import deque

##### **Agent Class**

In [8]:
class Agent:
    def __init__(self, agent_id, x, y):
        self.id = agent_id # save agent id
        self.x = x # x coordinate
        self.y = y # y coordinate
        self.vx = 0 # x velosity
        self.vy = 0 # y velosity
        self.known_positions ={} # dictionary to store the last known positions from messages
        self.sensor_range = 5.0
        self.received_current_step = [] # buffer for messages recived during current tick(tick = one time step in the simulation)

    def get_position(self): # helper: return current position as tuple
        return (self.x, self.y) # return x and y


    def update_velocity(self): # choose a new velocity by random position
        self.vx = random.uniform(-1, 1) # set vx to random float between -1 to 1
        self.vy = random.uniform(-1, 1) # set yx to random float between -1 to 1   


    def move(self): # update the position according to velocity 
        self.x += self.vx # increament x by vx
        self.y += self.vy # increament y by vy


    def sense_agents(self, agents): # This function lets an agent detect other agents near them even if communication fails
        sensed = []  # this is just an empty list, where we will store all nearby agents 
        for agent in agents: # iterate through all agents passsed in
            if agent.id == self.id: #skip sensing 
                continue # continue to next agent

            dist = math.dist(self.get_position(), agent.get_position()) # compute Educlidean distance(it's normal straight-line distance between two points)
            if dist <= self.sensor_range: # if within sensor range
                sensed.append((agent.id, agent.get_position())) # append (id, pos) from get_position(self)
        return sensed   # return list of sensed agents 
        

    def create_message(self):  # prepare a message to be send 
        return {  # return a dictionary representing the message
            "from": self.id, # sender id
            "pos": (self.x, self.y), # current position
            "intent": (self.vx, self.vy) # intended movement vector
        }
    

    def receive_message(self, msg): # call when a message is delevered to the agent
        self.received_current_step.append(msg) # append message to pre-tickbuffer. When messages arrive during the tick, we temporarily store them in self.received_this_step (buffer), before we process them

    
    def process_received(self): # process messages buffered during the tick(step)
        for msg in self.received_current_step: # iterate reicived messages
            sender_id = msg["from"] # extract sender id
            self.known_positions[sender_id] = msg["pos"] # update last-known position map
        self.received_current_step = [] # clear buffer after processing. (if you don’t clear it:messages from the previous tick will stay forever,the agent will think it received 1000 messages)


    def decide_action(self):  # decide action based on available information
        if not self.known_positions: # if no communications known this tick(step) 
            self.vx *= 0.2 # slow down as safety fallback (reduce vx)            
            self.vy *= 0.2 # slow down as safety fallback (reduce vy)
        








#### **Network Simulator(handles message delays)**

In [9]:
class Network:
    def __init__(self, network_type): 
        self.network_type = network_type # save what network is using urban, subarban, rural
        # Set network behavior based on type
        if network_type == "urban":
            self.drop_prob = 0.05 # 5% messages lost
            self.delay_range = (1,3) # messages arrive for 1-3 steps(small dalay)

        elif network_type == "subarban":
            self.drop_prob = 0.10 # 10% message are lost
            self.delay_range = (2,6) # message arrive for 2-6 steps(midium delay)


        elif network_type == "rural":
            self.drop_prob = 0.30 # 30% message loss
            self.delay_range = (5,15) # messages arrive for 5-15 steps(long delay)

        else:
            raise ValueError("Invalid Network Settings") # error if wrong network name is used
        

        #Message Queue
        self.queue = [] # list to store messages waiting to be delivered
        self.time = 0 # keeps track of the current simulation time

        self.comm_success = 0 # how many messages were delivered successfully
        self.comm_attempts = 0 # how many messaeges were attempted to sent
        self.collisions = 0 # how many collisions happend


        

#### **This function simulates sending messages.**

In [10]:
def broadcast(self, sender, msg, agents):
    for agent in agents:  # loop through all agents
        if agent.id == sender.id: # skip sending a message to ourselves
            continue

        self.comm_attempts += 1 # count that a message was attempted

        if random.random() < self.drop_prob:
            continue  # message is lost, do not deliver it

        delay = random.randint(*self.delay_range) # choose random delay with allowed range
        deliver_time = self.time + delay  # time when message should arrive

#### **Deliver Messages That Are Ready**

In [ ]:
def deliver_messages(self):
    ready = [item for itme in self.queue if item[0] <= self.time] 

    for deliver_time, agent, msg in ready:
        agent.receive_message(msg) # give the message to the agent
        self.comm_success += 1  # count successful delivery
        self.queue.remove((deliver_time, agent, msg)) # remove from queue

#### **ENVIRONMENT CLASS**

In [ ]:
class Environment:
    def __init__(self, num_agents, network_type, steps=100):
        self.agents = [
            Agent(
                agent_id=i,  # give each agent ID
                x=random.uniform(0, 50), # random starting x position
                y=random.uniform(0, 50) # random y position
            )
            for i in range(num_agents) # reapeat for all agents
        ]

                self.network = Network(network_type)
                self.steps = steps
